En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
# Enable automatic reloading of modules in Jupyter Notebook (improves development workflow)
%reload_ext autoreload

# Automatically restart the kernel whenever the source code changes
# (Provides a clean development environment)
%autoreload 2

print("IPython autoreload activated")


IPython autoreload activated


In [7]:
cd /content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project

/content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project


In [8]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Librerías e infraestructura necesaria
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 31.0 MB/s eta 0:00:00


In [10]:
import gdown
import zipfile

# ID del archivo (extraído del enlace)
file_id = "1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis"
# URL de descarga
download_url = f"https://drive.google.com/uc?id={file_id}"
# Nombre del archivo descargado
zip_output = "tweets.json.zip"

# Descargar el archivo
gdown.download(download_url, zip_output, quiet=False)

# Descomprimir el archivo
with zipfile.ZipFile(zip_output, 'r') as zip_ref:
    zip_ref.extractall("tweets_extracted")

print("Archivo descomprimido en la carpeta 'tweets_extracted'")

Downloading...
From (original): https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis
From (redirected): https://drive.google.com/uc?id=1ig2ngoXFTxP5Pa8muXo02mDTFexZzsis&confirm=t&uuid=d64ab3bb-1069-4fda-a701-8e139aef349c
To: /content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project/tweets.json.zip
100%|██████████| 60.4M/60.4M [00:01<00:00, 45.0MB/s]


Archivo descomprimido en la carpeta 'tweets_extracted'


Se importan los modulos python junto con sus funciones  

In [11]:
%run src/q1_memory.py
%run src/q1_time.py
%run src/q2_memory.py
%run src/q2_time.py
%run src/q3_memory.py
%run src/q3_time.py
%run src/utilidades.py


Se definen variables de entorno

In [12]:
# define la ruta
file_directory = '/content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project/tweets_extracted/'

# define nombre de archivo
file_name = 'farmers-protest-tweets-2021-2-4.json'

# concatenamos la ruta y el nombre del archivo
file_path = file_directory + file_name
print(file_path)

/content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project/tweets_extracted/farmers-protest-tweets-2021-2-4.json


Ejecución de la llamada de cantidad de twits en un periodo "q1_time"

In [38]:
# Usar line_profiler y memory_profiler
import line_profiler
from memory_profiler import profile  # Decorador para perfilar memoria
from memory_profiler import memory_usage

@time_it
def time_call(profiler, function, file_path):
    """
    Perfila el tiempo de ejecución de la función especificada usando line_profiler y devuelve su resultado.

    Args:
        profiler (LineProfiler): Instancia de line_profiler para perfilar la función.
        function (callable): Función a perfilar.
        file_path (str): Ruta del archivo que procesará la función.

    Returns:
        Any: Resultado de la función perfilada.
    """
    # Agregar explícitamente la función al perfilador
    profiler.add_function(function)

    # Usar un diccionario para capturar las variables locales
    local_vars = {"file_path": file_path, "result": None, "function": function}

    # Ejecutar y perfilar la función
    profiler.runctx("result = function(file_path)", globals(), local_vars)

    # Mostrar estadísticas del perfilador
    print("=== Estadísticas de Line Profiler ===")
    profiler.print_stats()
    print("=== Fin de las Estadísticas ===")

    # Retornar el resultado de la función
    return local_vars["result"]

@time_it
def memory_call(function, file_path):
    """
    Perfila el uso de memoria de la función especificada y devuelve su resultado.

    Args:
        function_name (callable): Función a perfilar.
        file_path (str): Ruta del archivo que procesará la función.

    Returns:
        Any: Resultado de la función perfilada.
    """
    def wrapper():
        return function(file_path)

    # Perfilar memoria usando memory_usage
    mem_usage, result = memory_usage(wrapper, retval=True, interval=0.1)

    # Mostrar estadísticas de memoria
    print("=== Estadísticas de Memory Profiler ===")
    print(f"Uso máximo de memoria: {max(mem_usage):.2f} MiB")
    print(f"Uso inicial de memoria: {mem_usage[0]:.2f} MiB")
    print(f"Uso final de memoria: {mem_usage[-1]:.2f} MiB")
    print("=== Fin de las Estadísticas ===")

    return result


In [14]:
profiler = line_profiler.LineProfiler()

In [39]:
top_tweets_time = time_call(profiler, q1_time, file_path)
print(f"Resultado final: {top_tweets_time}")

=== Estadísticas de Line Profiler ===
Timer unit: 1e-09 s

Total time: 38.9569 s
File: /content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project/src/q1_time.py
Function: q1_time at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def q1_time(file_path: str) -> List[Tuple[datetime.date, str, int]]:
     7         2       9989.0   4994.5      0.0      tweet_count_by_date_user = defaultdict(lambda: defaultdict(int))
     8                                           
     9                                               # Usamos chunksize para leer el archivo en trozos más pequeños
    10         2       2489.0   1244.5      0.0      chunksize = 10000  # Por ejemplo, leemos 10,000 líneas a la vez
    11        26        2e+10    9e+08     60.4      for chunk in pd.read_json(file_path, lines=True, chunksize=chunksize):
    12                                                   # Convertir la columna 'date' a datetime
    

Ejecución de llamada top emojis - 10 mas usados

In [40]:
# Call q2_time and print the result
top_emojis_time_valor = time_call(profiler, q2_time, file_path)
print(f"Resultado final: {top_emojis_time_valor}")

=== Estadísticas de Line Profiler ===
Timer unit: 1e-09 s

Total time: 38.9569 s
File: /content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project/src/q1_time.py
Function: q1_time at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def q1_time(file_path: str) -> List[Tuple[datetime.date, str, int]]:
     7         2       9989.0   4994.5      0.0      tweet_count_by_date_user = defaultdict(lambda: defaultdict(int))
     8                                           
     9                                               # Usamos chunksize para leer el archivo en trozos más pequeños
    10         2       2489.0   1244.5      0.0      chunksize = 10000  # Por ejemplo, leemos 10,000 líneas a la vez
    11        26        2e+10    9e+08     60.4      for chunk in pd.read_json(file_path, lines=True, chunksize=chunksize):
    12                                                   # Convertir la columna 'date' a datetime
    

Ejecución de llamada de tweets con más reacciones

In [41]:
top_mentions_time = time_call(profiler, q3_time, file_path)
print(f"Resultado final: {top_mentions_time}")

=== Estadísticas de Line Profiler ===
Timer unit: 1e-09 s

Total time: 38.9569 s
File: /content/drive/MyDrive/tw-latam-test-b/tw-latam-test-project/src/q1_time.py
Function: q1_time at line 6

Line #      Hits         Time  Per Hit   % Time  Line Contents
     6                                           def q1_time(file_path: str) -> List[Tuple[datetime.date, str, int]]:
     7         2       9989.0   4994.5      0.0      tweet_count_by_date_user = defaultdict(lambda: defaultdict(int))
     8                                           
     9                                               # Usamos chunksize para leer el archivo en trozos más pequeños
    10         2       2489.0   1244.5      0.0      chunksize = 10000  # Por ejemplo, leemos 10,000 líneas a la vez
    11        26        2e+10    9e+08     60.4      for chunk in pd.read_json(file_path, lines=True, chunksize=chunksize):
    12                                                   # Convertir la columna 'date' a datetime
    

In [44]:
top_emojis_memory = memory_call(q1_memory, file_path)
print(f"Resultado final: {top_emojis_memory}")

=== Estadísticas de Memory Profiler ===
Uso máximo de memoria: 826.79 MiB
Uso inicial de memoria: 826.71 MiB
Uso final de memoria: 825.80 MiB
=== Fin de las Estadísticas ===
⏱️ Tiempo de ejecución de memory_call: 10.6981 segundos
Resultado final: [(datetime.date(2021, 2, 24), 'preetysaini321', 107), (datetime.date(2021, 2, 23), 'Surrypuria', 135), (datetime.date(2021, 2, 22), 'preetysaini321', 110), (datetime.date(2021, 2, 21), 'Surrypuria', 161), (datetime.date(2021, 2, 20), 'MangalJ23056160', 108), (datetime.date(2021, 2, 19), 'Preetm91', 267), (datetime.date(2021, 2, 18), 'neetuanjle_nitu', 195), (datetime.date(2021, 2, 17), 'RaaJVinderkaur', 185), (datetime.date(2021, 2, 16), 'jot__b', 133), (datetime.date(2021, 2, 15), 'jot__b', 134), (datetime.date(2021, 2, 14), 'rebelpacifist', 119), (datetime.date(2021, 2, 13), 'MaanDee08215437', 178), (datetime.date(2021, 2, 12), 'RanbirS00614606', 176)]


In [45]:
top_mentions_memory = memory_call(q2_memory, file_path)
print(f"Resultado final: {top_mentions_memory}")

=== Estadísticas de Memory Profiler ===
Uso máximo de memoria: 824.83 MiB
Uso inicial de memoria: 824.82 MiB
Uso final de memoria: 824.83 MiB
=== Fin de las Estadísticas ===
⏱️ Tiempo de ejecución de memory_call: 11.2332 segundos
Resultado final: [('🙏', 7286), ('😂', 3072), ('🚜', 2972), ('✊', 2411), ('🌾', 2363), ('🏻', 2080), ('❤', 1779), ('🤣', 1668), ('🏽', 1218), ('👇', 1108)]


In [46]:
top_mentions_time = memory_call(q3_memory, file_path)
print(f"Resultado final: {top_mentions_time}")

=== Estadísticas de Memory Profiler ===
Uso máximo de memoria: 822.86 MiB
Uso inicial de memoria: 822.86 MiB
Uso final de memoria: 822.86 MiB
=== Fin de las Estadísticas ===
⏱️ Tiempo de ejecución de memory_call: 5.8867 segundos
Resultado final: [('@narendramodi', 2224), ('@Kisanektamorcha', 1829), ('@RakeshTikaitBKU', 1611), ('@PMOIndia', 1395), ('@RahulGandhi', 1106), ('@GretaThunberg', 1007), ('@RaviSinghKA', 1002), ('@UNHumanRights', 958), ('@rihanna', 951), ('@meenaharris', 915)]
